In [39]:
import numpy as np
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

def load_training_set(filename):
    with open(filename, "r", encoding='utf-8') as f:
        return [[x for x in row] for row in csv.reader(f, delimiter=',')]

#data = load_training_set("movie_metadata.csv")

data = pd.read_csv('movie_metadata.csv')

str_list = []

for name, value in data.iteritems():
    if type(value[1]) == str:
         str_list.append(name)
            
num_list = data.columns.difference(str_list)
movie_num = data[num_list]
#print(np.sum(movie_num.isnull()))

movie_num = movie_num.fillna(value=0, axis=1)
ratings = movie_num['imdb_score'].values
del movie_num['imdb_score']
#print(movie_num.head())

#ratings_integers = [int(round(rating)) for rating in ratings]

X = movie_num.values
# Data Normalization
X_std = StandardScaler().fit_transform(X)

# number_of_samples = len(ratings)
# print(number_of_samples)
# np.random.seed(0)
# random_indices = np.random.permutation(number_of_samples)
# num_training_samples = int(number_of_samples*0.75)
# print(num_training_samples)
# movie_num_training = X_std[random_indices[:num_training_samples]]
# ratings_cont_training = ratings[random_indices[:num_training_samples]]


# movie_num_test = X_std[random_indices[num_training_samples:]]
# ratings_cont_test = ratings[random_indices[num_training_samples:]]
# ratings_cont_training = list(ratings_cont_training)

# trainingdata:

number_for_training = int(round(0.75*len(ratings)))

movie_num_training = X_std[:number_for_training]
#ratings_int_training = ratings_integers[:number_for_training]
ratings_cont_training = ratings[:number_for_training]

#testdata:

movie_num_test = X_std[number_for_training:]
#ratings_int_test = ratings_integers[number_for_training:]
ratings_cont_test = ratings[number_for_training:]


In [40]:
# k-nearest neighbor met integers voor ratings (is niet heel accuraat):

# neighbor = KNeighborsClassifier(n_neighbors = 1)
# neighbor.fit(movie_num_training, ratings_int_training)
# neighbor.score(movie_num_test, ratings_int_test)


# k-nearest neighbor voor continue ratings:

regressor = KNeighborsRegressor()
regressor.fit(movie_num_training, ratings_cont_training)
y_predict_train = regressor.predict(movie_num_training)
y_predict_train = list(y_predict_train)

error = 0

for i in range(len(ratings_cont_test)):
    error += (abs(y_predict_train[i]-ratings_cont_training[i]))/ratings_cont_training[i]
training_error = error/len(ratings_cont_training)*100

print("Train error = "+'{}'.format(training_error)+" percent"+" in knn algorithm")

y_predict_test = regressor.predict(movie_num_test)
y_predict_test = list(y_predict_test)

error = 0

for i in range(len(ratings_cont_test)):
    error += (abs(y_predict_test[i]-ratings_cont_test[i]))/ratings_cont_test[i]
test_error = error/len(ratings_cont_test)*100

print("Test error = "'{}'.format(test_error)+" percent"+" in knn algorithm")

Train error = 2.7284311402050667 percent in knn algorithm
Test error = 17.103796448745108 percent in knn algorithm


In [41]:
# Neural networks:

